# Command line section

In [ ]:
import os

import pandas as pd

dir_in = 'E:\AI_Project\exp_vid1' # input directory
vid_in = 'exp_vid1' # input file
f_p_s = 1 # frames per second

### command #1 (images from video) ###########################################################
dir_cmd = dir_in.replace('\\', '/')
dir_out = dir_cmd + '/' + vid_in + '_imgs'
cmd_ffmpeg = 'ffmpeg -i '+dir_cmd+'/'+vid_in+'.mp4'+' -vf fps='+str(f_p_s)+' '+dir_cmd+'/'+vid_in+'_%04d.png'
os.system(cmd_ffmpeg) # command
print('Command for image generation executed:\n')

### command #2 (folder creation) #############################################################
os.mkdir(dir_out) # command
print('Folder for images created:\n')

### command #3 (moving images to folder) #####################################################
cmd_move_raw = 'move '+dir_cmd+'/*.png'+' '+dir_out
cmd_move = cmd_move_raw.replace('/', '\\')
os.system(cmd_move) # command
print('Images moved to the folder:\n')

### command #4 (creating the list of images as csv file) #####################################
csv_list_out = dir_cmd+'/'+vid_in+'_list'
csv_list_cmd = csv_list_out.replace('/', '\\')
dir_out_cmd=dir_out.replace('/', '\\')
write_dir= 'dir '+dir_out_cmd+' /b> '+csv_list_cmd+'.csv'
os.system(write_dir) # command
csv_in = csv_list_cmd+'.csv'
img_list_df = pd.read_csv(csv_in, header=None)
headerList=['File_name']
img_list_df.to_csv(csv_in, header=headerList, index=False)
print('File names list created')

# Data Extraction section

In [ ]:
import pandas as pd
import cv2

dir_im = dir_out.replace('\\', '/')

## reading csv file of image list ########################################################
img_list_df = pd.read_csv(csv_in)

## pre-allocation ########################################################################
imag_name = []
imag_time = []
x_1 = []
x_2 = []
x_mid = []

## looping over input images ##############################################################
end = img_list_df.shape
for i in range(end[0]):
    img_name = img_list_df.loc[i].at["File_name"]
    imag_name.append(img_name)
    ## extract time from image name #######################################################
    res = [j for j in range(len(img_name)) if img_name.startswith("_", j)]
    idx1 = res[1]
    idx2 = img_name.find(".")
    res = img_name[idx1+1: idx2]
    img_time = (int(res))/f_p_s
    imag_time.append(img_time)
    ## reading image ######################################################################
    in_img=dir_im + '/' + img_name
    image= cv2.imread(in_img)
    ## extracting position data from image ################################################
    original_image= image
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    edges= cv2.Canny(gray, 50,200)
    contours, hierarchy= cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= False)
    smallest_item= sorted_contours[0]
    M= cv2.moments(smallest_item)
    x,y,w,h= cv2.boundingRect(smallest_item)
    xcoordinate1= x
    x_1.append(xcoordinate1)
    xcoordinate2= x + w
    x_2.append(xcoordinate2)
    x_coord_mid = (xcoordinate2 + xcoordinate1)/2
    x_mid.append(x_coord_mid)

## saving extracted data to output csv file ###############################################
list_of_tuples = list(zip(imag_name, imag_time, x_1, x_2, x_mid))
df = pd.DataFrame(list_of_tuples, columns = ['Filename', 'Time', 'x-coord1', 'x-coord2', 'x-mid'])
csv_out = csv_list_cmd+'_out.csv'
df.to_csv(csv_out)
print("Output written to csv file successfully")

# extras

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# type(img_list_df.iloc[0]) # use of iloc for dataframe indxing
# print(img_list_df.loc[0].at["File_name"]) # use of loc for dataframe indexing
# img_list_np = img_list_df.to_numpy() # to convert a dataframe to a numpy array
